---
# <div align="center"><font color='green'> COSC 2673/2793 | Machine Learning  </font></div>
## <div align="center"> <font color='green'> **Example: Week08 Lecture QandA**</font></div>
---

# Feature Selection Demo

**Disclaimer: To simplify the example this code assumes the following without any investigations**

> A linear classifier is assumed to be appropriate.

> Hold out test set is used instead of cros-validation even though this might not be a good stratergy. 

> The obhective is to demostrate how the techniques are used and not to come up with the best model.


First lets load the libraries needed.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Lets now load the dataset. Uncomment appropriate part of the code
> Dataset 1 is a modified version of the `PIMA Indians diabetes dataset` (Missing values resolved). The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. 

> Dataset 2 is the 'Glaucoma' dataset where the goal is to predict if a patient has Glaucoma or not based on 63 different physiological measurements.

In [ ]:
# Dataset 1
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiaPedigreeFunction', 'Age', 'Class']
att_names = names[:-1]
dataframe = pd.read_csv(url, names=names)
dataframe.head(5)
s_features = 8

# # Dataset 2
# dataframe = pd.read_csv('./GlaucomaM.csv')
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# le.fit(dataframe['Class'])
# class_labels = le.inverse_transform([0,1])
# dataframe['Class'] = le.transform(dataframe['Class'])
# names = dataframe.columns
# att_names = names[:-1]
# dataframe.head(5)
# s_features = 40

Lets set up the data into attributes and target variable. 

In [ ]:
Y = dataframe['Class']
X = dataframe.drop(['Class'], axis = 1)
X.describe()

Lets first hold out some data for model evaluation. 70-30 split.

In [ ]:
from sklearn.model_selection import train_test_split
TrainX, TestX, TrainY, TestY = train_test_split(X, Y, test_size=0.3)

**Do normalization**

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(TrainX)

TrainX = preprocessing.scale(TrainX)
TestX = preprocessing.scale(TestX)

Now lets train a simple logistic regression model that use all the features. No regularaization (C = high value). 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

logReg = LogisticRegression(C=10000, max_iter=100, solver='liblinear')
logReg.fit(TrainX, TrainY)
pred = logReg.predict(TestX)

print("Test results for logistic regression with no feature selection")
print("\tF1 Score: ", f1_score(TestY, pred))
print("\tAccuracy: ", accuracy_score (TestY, pred))

## Filter methods - Example
Lets first try a filter method in sk-learn first. for this we are plannig to use chi-squared mesure to establisth the "best" features. The number of features to pick is set to 4. 

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif


featureSelector = SelectKBest(score_func=f_classif, k=5).fit(TrainX, TrainY)

plt.figure(figsize=(8,8))
scores = featureSelector.scores_
plt.xticks(rotation='vertical')
plt.barh(att_names, scores, )

# get the selected feature vectors
TrainX_new = featureSelector.transform(TrainX)
TestX_new = featureSelector.transform(TestX)

Lets now see how logistic regression works on selected features. 

In [ ]:
logReg = LogisticRegression(C=10000, max_iter=100, solver='liblinear')
logReg.fit(TrainX_new, TrainY)
pred = logReg.predict(TestX_new)

print("Test results for logistic regression with filtered feature selection")
print("\tF1 Score: ", f1_score(TestY, pred))
print("\tAccuracy: ", accuracy_score (TestY, pred))

How will performance chage wit the hyper-parameter k. Would you use cross valiadation?

In [ ]:
k_all = np.arange(1,s_features+1)
f1_hold = []
acc_hold = []
for k in k_all:
    featureSelector = SelectKBest(score_func=f_classif, k=k).fit(TrainX, TrainY)
    TrainX_new = featureSelector.transform(TrainX)
    TestX_new = featureSelector.transform(TestX)
    
    # Should consider CV
    logReg = LogisticRegression(C=10000, max_iter=100, solver='liblinear')
    logReg.fit(TrainX_new, TrainY)
    pred = logReg.predict(TestX_new)
    
    f1_hold.append(f1_score(TestY, pred))
    acc_hold.append(accuracy_score (TestY, pred))

plt.plot(k_all, f1_hold)
plt.plot(k_all, acc_hold)
plt.legend(['F1-score','Accuracy'])
plt.xlabel('Number of features selected')

## Wrapper methods - Example
Next lets implement `Recursive Feature Elimination` which is a type of wrapper feature selection method.

In [ ]:
from sklearn.feature_selection import RFE
model = LogisticRegression(C=10000, max_iter=100, solver='liblinear')
rfe = RFE(model, 5).fit(TrainX, TrainY)
print("Num Features: %s" % (rfe.n_features_))
sel_inx = np.ix_(rfe.support_)[0].tolist()
print("Selected Features: %s" % [att_names[i] for i in sel_inx])

# get the selected feature vectors
TrainX_new = rfe.transform(TrainX)
TestX_new = rfe.transform(TestX)

Lets now see how logistic regression works on selected features.

In [ ]:
logReg = LogisticRegression(C=10000, max_iter=100, solver='liblinear')
logReg.fit(TrainX_new, TrainY)
pred = logReg.predict(TestX_new)

print("Test results for logistic regression with Recursive Feature Elimination")
print("\tF1 Score: ", f1_score(TestY, pred))
print("\tAccuracy: ", accuracy_score (TestY, pred))

## Embedded methods - Example
Weel lets use l1 penalization. This is the Lasso penalty. Should use cross validation to set C. 

In [ ]:
logReg = LogisticRegression(penalty='l1', C=.1, max_iter=100, solver='liblinear')
logReg.fit(TrainX, TrainY)
pred = logReg.predict(TestX)

print("Test results for logistic regression with l1 penalty")
print("\tF1 Score: ", f1_score(TestY, pred))
print("\tAccuracy: ", accuracy_score (TestY, pred))

In [ ]:
plt.figure(figsize=(8,8))
coef = pd.Series(np.squeeze(logReg.coef_), index = att_names)
imp_coef = coef.sort_values()
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso Linear Model")
plt.show()

## Tree based feature selection - Feature Importance

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

tree_clf = ExtraTreesClassifier(n_estimators=100).fit(TrainX, TrainY)
scores = tree_clf.feature_importances_  
plt.figure(figsize=(8,8))
plt.xticks(rotation='vertical')
plt.barh(att_names, scores)

model = SelectFromModel(tree_clf, prefit=True)
TrainX_new = model.transform(TrainX)
TestX_new = model.transform(TestX)
print("Num Features: %s" % (TrainX_new.shape[1]))

tree_clf = ExtraTreesClassifier(n_estimators=100).fit(TrainX_new, TrainY)
pred = tree_clf.predict(TestX_new)

print("Test results for Random forest with tree feature selection")
print("\tF1 Score: ", f1_score(TestY, pred))
print("\tAccuracy: ", accuracy_score (TestY, pred))

# Feature construction with PCA

In [ ]:
from sklearn.decomposition import PCA
components = 5
pca = PCA(n_components=components).fit(TrainX)
TrainX_new = pca.transform(TrainX)
TestX_new = pca.transform(TestX)

logReg = LogisticRegression(C=10000, max_iter=100, solver='liblinear')
logReg.fit(TrainX_new, TrainY)
pred = logReg.predict(TestX_new)

print("Test results for logistic regression with PCA feature construction")
print("\tF1 Score: ", f1_score(TestY, pred))
print("\tAccuracy: ", accuracy_score (TestY, pred))

In [ ]:
print(pca.explained_variance_ratio_)
plt.bar(np.arange(1,components+1), pca.explained_variance_ratio_)
plt.show()

In [ ]:

plt.bar(np.arange(1,components+1), pca.singular_values_)
plt.show()

# Temporary Workings

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data[:, -2:]
Y = iris.target


In [ ]:
pca = PCA(n_components=2).fit(X)
X_new = pca.transform(X)

plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(X[:,0], X[:,1], 'ro')
plt.xlabel('Petal Length')
plt.ylabel('Petal Width')

plt.subplot(1,2,2)
plt.plot(X_new[:,0], X_new[:,1], 'ro')
plt.xlim([-4.5,4.5])
plt.ylim([-1.25,1.25])
plt.show()

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(X[:,0], X[:,1], 'ro')
plt.xlabel('Petal Length')
plt.ylabel('Petal Width')

plt.subplot(1,2,2)
plt.plot(X_new[:,0],np.zeros_like(X_new[:,0]), 'ro')
plt.xlim([-4.5,4.5])
plt.ylim([-1.25,1.25])
plt.show()